## Fine-Tuning an LLM for Movie Recommendations

#### Goals
- Improve the model's relevance in generating recommendations.
- Address challenges of domain-specific language and sparse user data.
- Create a deployable recommendation pipeline.

#### Requirements

- A pre-trained LLM (e.g., GPT-3, GPT-4, or an open-source model like GPT-J).
- A fine-tuning framework such as Hugging Face Transformers.
- A movie dataset (e.g., MovieLens or IMDb).
- Compute resources (GPU recommended).

#### Dataset preparation

**1. Define the Dataset**

Use a dataset containing:

- Movie descriptions (e.g., summaries, genres, metadata).
- User-item interaction data (e.g., ratings, reviews, watch history).

Input-Output Pair Formatting:

- Input: A prompt with user context, such as:

`User preferences: [list of liked movies]. Recommend 3 movies based on their taste.`

- Output: A list of relevant recommendations or a response explaining the recommendations.

**2. Preprocess the Data **

- Clean and deduplicate entries.
- Tokenize using a tokenizer matching the pre-trained LLM.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt-4")
data["input"] = data["user_preferences"] + data["movie_metadata"]
data["input"] = data["input"].apply(lambda x: tokenizer.encode(x, truncation=True, padding="max_length"))


#### Fine-Tuning Workflow

** 1. Set Up the Environment **

In [ ]:
pip install transformers datasets accelerate


** 2. Load Pre-Trained LLM and Dataset**


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import datasets

# Load pre-trained LLM
model = AutoModelForCausalLM.from_pretrained("gpt-4")
tokenizer = AutoTokenizer.from_pretrained("gpt-4")

# Load dataset
from datasets import load_dataset
dataset = load_dataset("movie_lens", split="train")

# Prepare input-output pairs
def format_prompt(example):
    return {
        "input": f"User preferences: {example['liked_movies']}. Recommend top movies.",
        "output": example["recommended_movies"]
    }

formatted_dataset = dataset.map(format_prompt)


**3. Define Training Configuration and fine-tune the model**


In [ ]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["validation"],
    tokenizer=tokenizer
)
trainer.train()


** 4. Evaluate the model **

In [ ]:
# Generate recommendations
prompts = [
    "User preferences: ['Inception', 'Interstellar', 'The Matrix']. Recommend movies."
]
inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(**inputs)

# Decode and print results
for i, prompt in enumerate(prompts):
    print(f"Prompt: {prompt}")
    print(f"Recommendation: {tokenizer.decode(outputs[i], skip_special_tokens=True)}")


#### Deployment and Usage
** Export the Model **

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

**Integrate into a Pipeline**

In [ ]:
from fastapi import FastAPI
from transformers import pipeline
import uvicorn

# Initialize the FastAPI app
app = FastAPI()

# Load the fine-tuned recommendation pipeline
recommendation_pipeline = pipeline("text-generation", model="./fine_tuned_model")

@app.post("/recommend")
def recommend(user_preferences: str):
    """
    Endpoint to generate movie recommendations based on user preferences.

    Args:
        user_preferences (str): A string describing the user's movie preferences.

    Returns:
        dict: A dictionary containing the generated recommendations.
    """
    prompt = f"User preferences: {user_preferences}. Recommend movies."
    response = recommendation_pipeline(prompt, max_length=150)
    return {"recommendations": response[0]["generated_text"]}

def main():
    """
    Entry point for running the FastAPI server.
    """
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)

if __name__ == "__main__":
    main()

#### Iterative Improvement
** Evaluation Metrics **
Evaluate using:

- Precision@K, Recall@K, and NDCG.
- Cosine similarity with pre-trained embeddings to assess semantic alignment.

**Automated Evaluation**

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")
generated_recommendations = ["The Dark Knight", "Tenet", "Avatar"]
ground_truth = ["The Dark Knight", "Tenet", "Inception"]

# Compute cosine similarity
gen_embeddings = model.encode(generated_recommendations)
truth_embeddings = model.encode(ground_truth)
cosine_score = util.cos_sim(gen_embeddings, truth_embeddings)
print("Cosine Similarity:", cosine_score)
